In [29]:
import torch
import torch.nn as nn
from torchsummary import summary

from PIL import Image

---
# ResNet (2015)
---

---
##### Basic Block
---

<img src="https://production-media.paperswithcode.com/methods/resnet-e1548261477164_2_mD02h5A.png" width="500" height="250"/>

In [30]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1 ,downsampling = False):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        if downsampling:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
                )
        else:
            downsample = None
        self.downsample = downsample

    def forward(self,x):
        i = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)

        if self.downsample is not None:
            i = self.downsample(i)

        x+=i
        x = self.relu(x)

        return x

---
##### Bottleneck
---

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTWVMFtZ_ob861NdYsCf2ddOCqxFXdC4WDeFQ&usqp=CAU" width="500" height="300"/>

In [31]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self,in_channels,out_channels,stride=1,downsampling=False):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()

        if downsampling:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion*out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*out_channels)
            )
        else:
            downsample =None
        self.downsample = downsample

    def forward(self,x):
        i = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.downsample is not None:
            i = self.downsample(i)

        x += i
        x = self.relu(x)
        return x

---
### ResNet18
---

![](https://blog.kakaocdn.net/dn/Bl6lG/btrDyFKASgY/LD9Z8BvHg1S76DcRe7yJd0/img.png)

In [32]:
class ResNet18(nn.Module):
    def __init__(self,classes):
        super().__init__()
        
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3 ,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
            )
        
        self.pooling_layer = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)

        self.basic_block1 = nn.Sequential(
            BasicBlock(in_channels=64,out_channels=64,stride=1,downsampling=False),
            BasicBlock(in_channels=64,out_channels=64,stride=1,downsampling=False)
            )
        
        self.basic_block2 = nn.Sequential(
            BasicBlock(in_channels=64,out_channels=128,stride=2,downsampling=True),
            BasicBlock(in_channels=128,out_channels=128,stride=1,downsampling=False)
        )

        self.basic_block3 = nn.Sequential(
            BasicBlock(in_channels=128,out_channels=256,stride=2,downsampling=True),
            BasicBlock(in_channels=256,out_channels=256,stride=1,downsampling=False)
        )

        self.basic_block4 = nn.Sequential(
            BasicBlock(in_channels=256,out_channels=512,stride=2,downsampling=True),
            BasicBlock(in_channels=512,out_channels=512,stride=1,downsampling=False)
        )

        self.avg_pooling_layer = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.output_layer = nn.Linear(512,classes)
    
    def forward(self,x):
        x = self.conv_layer1(x)
        x = self.pooling_layer(x)
        x = self.basic_block1(x)
        x = self.basic_block2(x)
        x = self.basic_block3(x)
        x = self.basic_block4(x)
        x = self.avg_pooling_layer(x)
        x = x.view(x.shape[0],-1)
        x = self.output_layer(x)
        return x


In [33]:
model = ResNet18(100)
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]           4,096
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

---
### ResNet34
----

![](https://miro.medium.com/v2/resize:fit:850/1*C8jf92MeHZnxnbpMkz6jkQ.png)

In [34]:
class ResNet34(nn.Module):
    def __init__(self,classes):
        super().__init__()
        
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3 ,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
            )
        
        self.pooling_layer = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)

        self.basic_block1 = nn.Sequential(
            BasicBlock(in_channels=64,out_channels=64,stride=1,downsampling=False),
            BasicBlock(in_channels=64,out_channels=64,stride=1,downsampling=False),
            BasicBlock(in_channels=64,out_channels=64,stride=1,downsampling=False)
            )
        
        self.basic_block2 = nn.Sequential(
            BasicBlock(in_channels=64,out_channels=128,stride=2,downsampling=True),
            BasicBlock(in_channels=128,out_channels=128,stride=1,downsampling=False),
            BasicBlock(in_channels=128,out_channels=128,stride=1,downsampling=False),
            BasicBlock(in_channels=128,out_channels=128,stride=1,downsampling=False)
        )

        self.basic_block3 = nn.Sequential(
            BasicBlock(in_channels=128,out_channels=256,stride=2,downsampling=True),
            BasicBlock(in_channels=256,out_channels=256,stride=1,downsampling=False),
            BasicBlock(in_channels=256,out_channels=256,stride=1,downsampling=False),
            BasicBlock(in_channels=256,out_channels=256,stride=1,downsampling=False),
            BasicBlock(in_channels=256,out_channels=256,stride=1,downsampling=False),
            BasicBlock(in_channels=256,out_channels=256,stride=1,downsampling=False)
        )

        self.basic_block4 = nn.Sequential(
            BasicBlock(in_channels=256,out_channels=512,stride=2,downsampling=True),
            BasicBlock(in_channels=512,out_channels=512,stride=1,downsampling=False),
            BasicBlock(in_channels=512,out_channels=512,stride=1,downsampling=False),
        )

        self.avg_pooling_layer = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.output_layer = nn.Linear(512,classes)
    
    def forward(self,x):
        x = self.conv_layer1(x)
        x = self.pooling_layer(x)
        x = self.basic_block1(x)
        x = self.basic_block2(x)
        x = self.basic_block3(x)
        x = self.basic_block4(x)
        x = self.avg_pooling_layer(x)
        x = x.view(x.shape[0],-1)
        x = self.output_layer(x)
        return x


In [35]:
model = ResNet34(2)
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]           4,096
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

---
### ResNet50
----

In [36]:
class ResNet50(nn.Module):
    def __init__(self,classes):
        super().__init__()
        
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3 ,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
            )
        
        self.pooling_layer = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)

        self.bottleneck_block1 = nn.Sequential(
            Bottleneck(in_channels=64,out_channels=64,stride=1,downsampling=True),
            Bottleneck(in_channels=256,out_channels=64,stride=1,downsampling=False),
            Bottleneck(in_channels=256,out_channels=64,stride=1,downsampling=False)
            )
        
        self.bottleneck_block2 = nn.Sequential(
            Bottleneck(in_channels=256,out_channels=128,stride=2,downsampling=True),
            Bottleneck(in_channels=512,out_channels=128,stride=1,downsampling=False),
            Bottleneck(in_channels=512,out_channels=128,stride=1,downsampling=False),
            Bottleneck(in_channels=512,out_channels=128,stride=1,downsampling=False)
        )

        self.bottleneck_block3 = nn.Sequential(
            Bottleneck(in_channels=512,out_channels=256,stride=2,downsampling=True),
            Bottleneck(in_channels=1024,out_channels=256,stride=1,downsampling=False),
            Bottleneck(in_channels=1024,out_channels=256,stride=1,downsampling=False),
            Bottleneck(in_channels=1024,out_channels=256,stride=1,downsampling=False),
            Bottleneck(in_channels=1024,out_channels=256,stride=1,downsampling=False),
            Bottleneck(in_channels=1024,out_channels=256,stride=1,downsampling=False),
        )

        self.bottleneck_block4 = nn.Sequential(
            Bottleneck(in_channels=1024,out_channels=512,stride=2,downsampling=True),
            Bottleneck(in_channels=2048,out_channels=512,stride=1,downsampling=False),
            Bottleneck(in_channels=2048,out_channels=512,stride=1,downsampling=False)
        )

        self.avg_pooling_layer = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.output_layer = nn.Linear(2048,classes)
    
    def forward(self,x):
        x = self.conv_layer1(x)
        x = self.pooling_layer(x)
        x = self.bottleneck_block1(x)
        x = self.bottleneck_block2(x)
        x = self.bottleneck_block3(x)
        x = self.bottleneck_block4(x)
        x = self.avg_pooling_layer(x)
        x = x.view(x.shape[0],-1)
        x = self.output_layer(x)
        return x


In [37]:
model = ResNet50(4)
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

---
### ResNet101
---

In [38]:
class ResNet101(nn.Module):
    def __init__(self,classes):
        super().__init__()
        
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3 ,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
            )
        
        self.pooling_layer = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)

        self.bottleneck_block1 = nn.Sequential(
            Bottleneck(in_channels=64,out_channels=64,stride=1,downsampling=True),
            Bottleneck(in_channels=256,out_channels=64,stride=1,downsampling=False),
            Bottleneck(in_channels=256,out_channels=64,stride=1,downsampling=False)
            )
        
        self.bottleneck_block2 = nn.Sequential(
            Bottleneck(in_channels=256,out_channels=128,stride=2,downsampling=True),
            Bottleneck(in_channels=512,out_channels=128,stride=1,downsampling=False),
            Bottleneck(in_channels=512,out_channels=128,stride=1,downsampling=False),
            Bottleneck(in_channels=512,out_channels=128,stride=1,downsampling=False)
        )

        block3_configuration = list()
        for iteration in range(23):
            if iteration == 0:
                block3_configuration.append(Bottleneck(in_channels=512,out_channels=256,stride=2,downsampling=True))
            else:
                block3_configuration.append(Bottleneck(in_channels=1024,out_channels=256,stride=1,downsampling=False))

        self.bottleneck_block3 = nn.Sequential(*block3_configuration)

        self.bottleneck_block4 = nn.Sequential(
            Bottleneck(in_channels=1024,out_channels=512,stride=2,downsampling=True),
            Bottleneck(in_channels=2048,out_channels=512,stride=1,downsampling=False),
            Bottleneck(in_channels=2048,out_channels=512,stride=1,downsampling=False)
        )

        self.avg_pooling_layer = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.output_layer = nn.Linear(2048,classes)
    
    def forward(self,x):
        x = self.conv_layer1(x)
        x = self.pooling_layer(x)
        x = self.bottleneck_block1(x)
        x = self.bottleneck_block2(x)
        x = self.bottleneck_block3(x)
        x = self.bottleneck_block4(x)
        x = self.avg_pooling_layer(x)
        x = x.view(x.shape[0],-1)
        x = self.output_layer(x)
        return x


In [39]:
model = ResNet101(1000)
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

---
### ResNet152
---

In [40]:
class ResNet152(nn.Module):
    def __init__(self,classes):
        super().__init__()
        
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3 ,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
            )
        
        self.pooling_layer = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)

        self.bottleneck_block1 = nn.Sequential(
            Bottleneck(in_channels=64,out_channels=64,stride=1,downsampling=True),
            Bottleneck(in_channels=256,out_channels=64,stride=1,downsampling=False),
            Bottleneck(in_channels=256,out_channels=64,stride=1,downsampling=False)
            )
        
        self.bottleneck_block2 = nn.Sequential(
            Bottleneck(in_channels=256,out_channels=128,stride=2,downsampling=True),
            Bottleneck(in_channels=512,out_channels=128,stride=1,downsampling=False),
            Bottleneck(in_channels=512,out_channels=128,stride=1,downsampling=False),
            Bottleneck(in_channels=512,out_channels=128,stride=1,downsampling=False),
            Bottleneck(in_channels=512,out_channels=128,stride=1,downsampling=False),
            Bottleneck(in_channels=512,out_channels=128,stride=1,downsampling=False),
            Bottleneck(in_channels=512,out_channels=128,stride=1,downsampling=False),
            Bottleneck(in_channels=512,out_channels=128,stride=1,downsampling=False)
        )

        block3_configuration = list()
        for iteration in range(36):
            if iteration == 0:
                block3_configuration.append(Bottleneck(in_channels=512,out_channels=256,stride=2,downsampling=True))
            else:
                block3_configuration.append(Bottleneck(in_channels=1024,out_channels=256,stride=1,downsampling=False))

        self.bottleneck_block3 = nn.Sequential(*block3_configuration)


        self.bottleneck_block4 = nn.Sequential(
            Bottleneck(in_channels=1024,out_channels=512,stride=2,downsampling=True),
            Bottleneck(in_channels=2048,out_channels=512,stride=1,downsampling=False),
            Bottleneck(in_channels=2048,out_channels=512,stride=1,downsampling=False)
        )

        self.avg_pooling_layer = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.output_layer = nn.Linear(2048,classes)
    
    def forward(self,x):
        x = self.conv_layer1(x)
        x = self.pooling_layer(x)
        x = self.bottleneck_block1(x)
        x = self.bottleneck_block2(x)
        x = self.bottleneck_block3(x)
        x = self.bottleneck_block4(x)
        x = self.avg_pooling_layer(x)
        x = x.view(x.shape[0],-1)
        x = self.output_layer(x)
        return x


In [41]:
model = ResNet152(1000)
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [42]:
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR100

from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

ImportError: cannot import name 'type_before_parametrizations' from 'torch.nn.utils.parametrize' (c:\ProgramData\Anaconda3\envs\dl\lib\site-packages\torch\nn\utils\parametrize.py)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('DEVICE:',device)

DEVICE: cpu


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])

train_set = CIFAR100(root=r'C:\Users\Administrator\Desktop\Dataset',
                           transform=transform,
                           train=True,
                           download=True
                           )
test_set = CIFAR100(root=r'C:\Users\Administrator\Desktop\Dataset',
                           transform=transform,
                           train=False,
                           download=True
                           )

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_loader = DataLoader(dataset=train_set,
                          batch_size=32,
                          shuffle=True,
                          drop_last=True)

test_loader = DataLoader(dataset=test_set,
                         drop_last=True,
                         shuffle=True,
                         batch_size=32)

In [ ]:
model = ResNet18(100).to(device)

num_epochs = 30
batch_size = 128
num_train_batchs = len(train_loader)
num_test_batchs = len(test_loader)

loss_func = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
#학습 진행
for epoch in tqdm(range(num_epochs)):
    model.train()
    
    epoch_loss = 0.0
    epoch_correct_count = 0

    for data in tqdm(train_loader):
        optimizer.zero_grad()

        imgs,labels = data
        imgs,labels = imgs.to(device), labels.to(device)

        outputs = model(imgs)
        batch_loss = loss_func(outputs,labels)

        epoch_loss += batch_loss/num_train_batchs
        epoch_correct_count += (outputs.argmax(dim=-1)==labels).sum().item()
        
        batch_loss.backward()
        optimizer.step()
    epoch_accuracy = epoch_correct_count/len(train_set)*100
    
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        val_correct_count = 0

        for data in test_loader:
            val_imgs, val_labels = data
            val_imgs, val_labels = val_imgs.to(device), val_labels.to(device)

            val_outputs = model(val_imgs)
            val_batch_loss = loss_func(val_outputs,val_labels)
            val_loss += val_batch_loss/num_test_batchs

            val_correct_count += (val_outputs.argmax(dim=-1)==val_labels).sum().item()
        val_accuracy = val_correct_count/len(test_set)*100

    print(f'[EPOCH {epoch+1:2}/{num_epochs:2}] [TRAIN LOSS: {epoch_loss:5f}] [TRAIN ACCURACY: {epoch_accuracy:2f}%] [VAL LOSS: {val_loss:5f}] [VAL ACCURACY: {val_accuracy:2f}%]')

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/1562 [00:00<?, ?it/s]

KeyboardInterrupt: 